<h1 style = "color : #0EE071; text-align : center;"><em>Nata Project</em> - Data Preprocessing Notebook</h1>
<p style = "font-size : 16px; text-align: center;">This notebook has the funcion of preparing the <code>Nata_Files/learn.csv</code> dataset for the learning model.</p>
<br>
<p style = "font-size : 12px; text-align: center;"><b>NOVA IMS</b></p>
<p style = "font-size : 10px; text-align: center;">Machine Learning I</p>
<p style = "font-size : 10px; text-align: center;">Diogo Gonçalves, João Marques, Juan Mendes, Gustavo Franco & Lucas Casimiro</p>
<br>

CENAS PA NAO ESQUECER NO 1 NOTEBOOK
- mostrar quais as strings unicas pas colunas origin e pastry_type
- outliers
- missing values
- datatypes das colunas (nao corrigi nenhum pq ns se e preciso)
- fazer o iqr pa identificar outliers (ainda n limpei nenhum q n os obvios por causa disso)
- aprender a tratar da skewness tp log transformations


<h2  style = "color : #0EE071;"> Imports</h2>

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

<hr style = "border: 3px solid #0EE071;">
<h2 style = "color : #0EE071;">Dataset Reading </h2>
<p style = "font-size : 15px;">Reading of dataset from <code>learn.csv</code> file, equal to the last document, and making a copy where we will preprocess the data, so we can preserve the original dataset.</p>

In [28]:
learn_data = pd.read_csv("Nata_files/learn.csv", sep = ",", index_col = 0)
learn_data_copy = learn_data.copy()

<hr style = "border: 3px solid #0EE071;">
<h2 style = "color : #0EE071;">Data Cleaning </h2>
<p style = "font-size : 15px;">Handling any outliers, missing values or troublesome data points identified in NB1.</p>

First, converting 'origin' and 'pastry_type' values to standardized texts (e.g.,turning 'Lisboa', 'LISBOA', 'lisboa' into the same value), which was recognized as a problem in NB1.

In [29]:
learn_data_copy['pastry_type'].unique()

array(['Pastel Nata', nan, 'Pastel de Nata', 'Pastel de nata',
       'Pastel De Nata'], dtype=object)

In [30]:
# Turning every string in 'origin' and 'pastry_type' columns to lowercase and stripping whitespace
learn_data_copy['origin'] = learn_data_copy['origin'].astype(str).str.lower().str.strip()
learn_data_copy['pastry_type'] = learn_data_copy['pastry_type'].astype(str).str.lower().str.strip()

# Solving the problem 
learn_data_copy['pastry_type'] = learn_data_copy['pastry_type'].replace({'pastel nata': 'pastel de nata',
                                                                         'nan': np.nan})

<h3 style="color: #0EE071;">Handling Outliers</h3>

Starting by removing the physically impossible values identified in NB1.

In [31]:
# Setting these values to NaN instead of removing the rows, to avoid losing too much data

# Sugar content cannot exceed 100g per 100g
learn_data_copy.loc[learn_data_copy['sugar_content'] > 100, 'sugar_content'] = np.nan

# Fat percentage cannot exceed 100%
learn_data_copy.loc[learn_data_copy['cream_fat_content'] > 100, 'cream_fat_content'] = np.nan

# Salt > 100g per kg is inedible 
learn_data_copy.loc[learn_data_copy['salt_ratio'] > 100, 'salt_ratio'] = np.nan

# Eggs cook at ~65C. 170C or 575C is impossible for raw egg addition
learn_data_copy.loc[learn_data_copy['egg_temperature'] > 100, 'egg_temperature'] = np.nan

# Oven/Final temp > 500C is likely an error 
learn_data_copy.loc[learn_data_copy['final_temperature'] > 500, 'final_temperature'] = np.nan
learn_data_copy.loc[learn_data_copy['oven_temperature'] > 500, 'oven_temperature'] = np.nan

<h3 style="color: #0EE071;">Handling Missing Values</h3>

As seen in NB1, the 'notes_baker' column has no values, so it will be dropped.

In [32]:
learn_data_copy.drop('notes_baker', axis=1, inplace=True)

As seen in the correlation heatmap in NB1, we see 'oven_temperature' and 'final_temperature' have an extremely high correlation, which means having both is redundant, so 'final_temperature' will be dropped.

In [33]:
learn_data_copy.drop('final_temperature', axis=1, inplace=True)

There is a missing value in the 'quality_class' column, which is the target variable, so we will drop that whole row as it won't be used for the model.

In [34]:
learn_data_copy.dropna(subset=['quality_class'], inplace=True)

For the remaining columns, we will use either mean or median for numerical columns (depending on skewness), and use the mode for categorical variables.

In [39]:
# Identifying for numerical columns
num_cols = learn_data_copy.select_dtypes(include=['float64'])

# Using mean or median to fill missing values in numerical columns based on skewness
for col in num_cols:
    skewness = learn_data_copy[col].skew()

    # If skewness is between -0.5 and 0.5, we consider it low skewness
    if abs(skewness) < 0.5:
        # If skewness is low, use mean
        method = 'mean'
        learn_data_copy[col] = learn_data_copy[col].fillna(learn_data_copy[col].mean())

    else:
        # If skewness is high, use median
        method = 'median'
        learn_data_copy[col] = learn_data_copy[col].fillna(learn_data_copy[col].median())

#Clarifying which method was used for each column, along with the skewness value
    print(f'{col} - Skewness: {skewness}')
    print(f' - Missing values filled using {method}')

ambient_humidity - Skewness: 0.015582878844407105
 - Missing values filled using mean
baking_duration - Skewness: 1.5361731965648864
 - Missing values filled using median
cooling_period - Skewness: 0.31479598704826645
 - Missing values filled using mean
cream_fat_content - Skewness: -0.8264821399325748
 - Missing values filled using median
egg_temperature - Skewness: -0.0215523837906362
 - Missing values filled using mean
egg_yolk_count - Skewness: 0.5498558732641039
 - Missing values filled using median
lemon_zest_ph - Skewness: 0.33604261530135693
 - Missing values filled using mean
oven_temperature - Skewness: -0.06195219451506645
 - Missing values filled using mean
preheating_time - Skewness: 1.6861674628892802
 - Missing values filled using median
salt_ratio - Skewness: 0.8337125217634596
 - Missing values filled using median
sugar_content - Skewness: 1.307228097089835
 - Missing values filled using median
vanilla_extract - Skewness: 1.6408803255644309
 - Missing values filled usi

In [ ]:
# Using the mode to fill missing values in categorical columns
cat_cols = ['origin', 'pastry_type']
for col in cat_cols:
    learn_data_copy[col] = learn_data_copy[col].fillna(learn_data_copy[col].mode()[0])

<hr style = "border: 3px solid #0EE071;">
<h2 style = "color : #0EE071;">Encoding categorical variables </h2>
<p style = "font-size : 15px;">Turning the values of the columns 'origin' and 'quality_class' into numbers, so they can be used in the model.</p>

For the 'origin' column (feature), since there are only 2 values after the standardization ('lisboa' and 'porto'), we will use One-Hot Encoding (Binary Encoding).

In [ ]:
# drop_first=True creates just one column instead of two
learn_data_copy = pd.get_dummies(data=learn_data_copy, columns = ['origin'], drop_first = True)

For the 'quality_class' columns, as it is a target variable, we will use Label Encoding.

In [ ]:
# Manually mapping it to ensure '1' is the positive class 'OK'
learn_data_copy['quality_class'] = learn_data_copy['quality_class'].map({'OK': 1, 'KO' : 0})

<hr style = "border: 3px solid #0EE071;">
<h2 style = "color : #0EE071;">Data Partitioning</h2>
<p style = "font-size : 15px;">Before we perform any scaling, we must split our dataset. This is crucial to prevent <b>Data Leakage</b>.</p>

In [ ]:
# Define X (features) and y (target)
X = learn_data_copy.drop('quality_class', axis=1)
y = learn_data_copy['quality_class']

# Split into training and testing sets, using stratify to maintain class distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Checking if the dataset is balanced in both sets
print("Training Set Class Distribution:")
print(y_train.value_counts(normalize=True))
print("\nTest Set Class Distribution:")
print(y_test.value_counts(normalize=True))

Training Set Class Distribution:
quality_class
1.0    0.635008
0.0    0.364992
Name: proportion, dtype: float64

Test Set Class Distribution:
quality_class
1.0    0.635577
0.0    0.364423
Name: proportion, dtype: float64


Note: We are performing a 2-way split (Train/Test). We will not create a validation set here. Instead, we'll use k-Fold Cross-Validation on the training set during the Modeling and Tuning phases. This ensures we maximize the data available for training while maintaining a robust evaluation protocol.

<hr style = "border: 3px solid #0EE071;">
<h2 style = "color : #0EE071;">Feature Scaling</h2>
<p style = "font-size : 15px;">To optimize our model, we must address the different scales and distributions of our data. In this section, we'll apply Standard Scaling to center all numerical variables at a mean of 0 and variance of 1. This prevents features with larger magnitudes from dominating the model's learning process.</p>


In [48]:
# Selecting only numerical columns for scaling
features_to_scale = X.select_dtypes(include=['float64']).columns

# Defining the scaler
scaler = StandardScaler()

# Fitting only the training data and transforming both 
X_train[features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
X_test[features_to_scale] = scaler.transform(X_test[features_to_scale])